In [1]:
import numpy as np
import pandas as pd

df = pd.read_csv('/kaggle/input/crop-recommendation-dataset/Crop_recommendation.csv')
df.head()

,N,P,K,temperature,humidity,ph,rainfall,label
0,90,42,43,20.879744,82.002744,6.502985,202.935536,rice
1,85,58,41,21.770462,80.319644,7.038096,226.655537,rice
2,60,55,44,23.004459,82.320763,7.840207,263.964248,rice
3,74,35,40,26.491096,80.158363,6.980401,242.864034,rice
4,78,42,42,20.130175,81.604873,7.628473,262.717340,rice


In [2]:
df.shape

(2200, 8)

In [3]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2200 entries, 0 to 2199
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   N            2200 non-null   int64  
 1   P            2200 non-null   int64  
 2   K            2200 non-null   int64  
 3   temperature  2200 non-null   float64
 4   humidity     2200 non-null   float64
 5   ph           2200 non-null   float64
 6   rainfall     2200 non-null   float64
 7   label        2200 non-null   object 
dtypes: float64(4), int64(3), object(1)
memory usage: 137.6+ KB
None


In [4]:
df["label"].value_counts()

label
rice           100
maize          100
jute           100
cotton         100
coconut        100
papaya         100
orange         100
apple          100
muskmelon      100
watermelon     100
grapes         100
mango          100
banana         100
pomegranate    100
lentil         100
blackgram      100
mungbean       100
mothbeans      100
pigeonpeas     100
kidneybeans    100
chickpea       100
coffee         100
Name: count, dtype: int64

In [5]:
rice = df.loc[df["label"] == 'rice']
rice

,N,P,K,temperature,humidity,ph,rainfall,label
0,90,42,43,20.879744,82.002744,6.502985,202.935536,rice
1,85,58,41,21.770462,80.319644,7.038096,226.655537,rice
2,60,55,44,23.004459,82.320763,7.840207,263.964248,rice
3,74,35,40,26.491096,80.158363,6.980401,242.864034,rice
4,78,42,42,20.130175,81.604873,7.628473,262.717340,rice
...,...,...,...,...,...,...,...,...
95,88,46,42,22.683191,83.463583,6.604993,194.265172,rice
96,93,47,37,21.533463,82.140041,6.500343,295.924880,rice
97,60,55,45,21.408658,83.329319,5.935745,287.576694,rice
98,78,35,44,26.543481,84.673536,7.072656,183.622266,rice


In [6]:
# Reduce the decimal places of the floating point variables to 2
df['temperature'] = df['temperature'].round(2)
df['humidity'] = df['humidity'].round(2)
df['ph'] = df['ph'].round(2)
df['rainfall'] = df['rainfall'].round(2)

In [7]:
df

,N,P,K,temperature,humidity,ph,rainfall,label
0,90,42,43,20.88,82.00,6.50,202.94,rice
1,85,58,41,21.77,80.32,7.04,226.66,rice
2,60,55,44,23.00,82.32,7.84,263.96,rice
3,74,35,40,26.49,80.16,6.98,242.86,rice
4,78,42,42,20.13,81.60,7.63,262.72,rice
...,...,...,...,...,...,...,...,...
2195,107,34,32,26.77,66.41,6.78,177.77,coffee
2196,99,15,27,27.42,56.64,6.09,127.92,coffee
2197,118,33,30,24.13,67.23,6.36,173.32,coffee
2198,117,32,34,26.27,52.13,6.76,127.18,coffee


In [13]:
# Create a dictionary to map labels to numbers
label_mapping = {label: i for i, label in enumerate(df['label'].unique())}

# Map the labels to numbers
df['label_num'] = df['label'].map(label_mapping)

print(label_mapping)

{'rice': 0, 'maize': 1, 'chickpea': 2, 'kidneybeans': 3, 'pigeonpeas': 4, 'mothbeans': 5, 'mungbean': 6, 'blackgram': 7, 'lentil': 8, 'pomegranate': 9, 'banana': 10, 'mango': 11, 'grapes': 12, 'watermelon': 13, 'muskmelon': 14, 'apple': 15, 'orange': 16, 'papaya': 17, 'coconut': 18, 'cotton': 19, 'jute': 20, 'coffee': 21}


In [15]:
print(df.head().to_markdown(index=False, numalign="left", stralign="left"))

| N   | P   | K   | temperature   | humidity   | ph   | rainfall   | label   | label_num   |
|:----|:----|:----|:--------------|:-----------|:-----|:-----------|:--------|:------------|
| 90  | 42  | 43  | 20.88         | 82         | 6.5  | 202.94     | rice    | 0           |
| 85  | 58  | 41  | 21.77         | 80.32      | 7.04 | 226.66     | rice    | 0           |
| 60  | 55  | 44  | 23            | 82.32      | 7.84 | 263.96     | rice    | 0           |
| 74  | 35  | 40  | 26.49         | 80.16      | 6.98 | 242.86     | rice    | 0           |
| 78  | 42  | 42  | 20.13         | 81.6       | 7.63 | 262.72     | rice    | 0           |


In [17]:
import json
# Save label mapping to a JSON file
with open('label_mapping.json', 'w') as f:
    json.dump(label_mapping, f)

In [21]:
from sklearn.model_selection import train_test_split
import joblib

# Select features (all columns except 'label' and 'label_num')
X = df.drop(['label', 'label_num'], axis=1) 

# Define target variable
y = df['label_num']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [22]:
from sklearn.ensemble import RandomForestClassifier

rfc=RandomForestClassifier()

rfc.fit(X_train, y_train)

rfc.score(X_test,y_test)

0.9931818181818182

In [23]:


from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

y_pred = rfc.predict(X_test)

# Calculating Precision, Recall, F1 Score, and Accuracy
precision = precision_score(y_test, y_pred, average='micro')
recall = recall_score(y_test, y_pred, average='micro')
f1 = f1_score(y_test, y_pred, average='micro')
accuracy = accuracy_score(y_test, y_pred)

# Formatting the results to 2 decimal places
formatted_precision = "{:.2f}".format(precision*100)
formatted_recall = "{:.2f}".format(recall*100)
formatted_f1 = "{:.2f}".format(f1*100)
formatted_accuracy = "{:.2f}".format(accuracy*100)

print(f'Precision: {formatted_precision}')
print(f'Recall: {formatted_recall}')
print(f'F1 Score: {formatted_f1}')
print(f'Accuracy: {formatted_accuracy}')



Precision: 99.32
Recall: 99.32
F1 Score: 99.32
Accuracy: 99.32


In [24]:
import joblib

# Assuming model is your trained random forest model
joblib.dump(rfc, 'crop_recommendation.joblib')

['crop_recommendation.joblib']